# EUMETSAT API Wrapper

<br>

### Imports

In [102]:
import pandas as pd

import dotenv
import zipfile
import os
import io

from requests.auth import HTTPBasicAuth
import requests

from IPython.display import JSON

<br>

### User Input

In [106]:
data_dir = '../data/raw'
env_vars_fp = '../.env'

<br>

### Authorising API Access

First we'll load the the environment variables and assign the user key and secret

In [103]:
dotenv.load_dotenv(env_vars_fp)

user_key = os.environ.get('user_key')
user_secret = os.environ.get('user_secret')

<br>

We'll then use to request an access token for the data download API

In [104]:
def retrieve_access_token(user_key, user_secret):
    token_url = 'https://api.eumetsat.int/token'

    data = {
      'grant_type': 'client_credentials'
    }

    r = requests.post(token_url, data=data, auth=(user_key, user_secret))
    access_token = r.json()['access_token']

    return access_token

access_token = retrieve_access_token(user_key, user_secret)

print(f'Access token: {access_token}')

Access token: 6235ec9f-c890-3784-8493-737d1950f211


<br>

### Querying Available Data

In [94]:
search_url = 'https://api.eumetsat.int/data/search-products/os'
 
params = {
    'format': 'json',
    'pi': 'EO:EUM:DAT:MSG:MSG15-RSS',
    'si': '0',
    'c': '20',
    'sort': 'start,time,0',
    'dtstart': '2020-10-11T00:00:00Z',
    'dtend': '2020-10-12T04:39:17Z'
}

r = requests.get(search_url, params=params)

assert r.ok, f'Request was unsuccesfull - Error code: {r.status_code}'

r_json = r.json()
JSON(r_json)

<IPython.core.display.JSON object>

In [97]:
data_id_to_link = lambda data_id: f'https://api.eumetsat.int/data/download/products/{data_id}'

data_ids = [ftr['id'] for ftr in r_json['features']]
data_links = [data_id_to_link(data_id) for data_id in data_ids]

example_data_link = data_links[0]

example_data_link

'https://api.eumetsat.int/data/download/products/MSG3-SEVI-MSG15-0100-NA-20201012043915.018000000Z-NA'

<br>

### Downloading Data

In [107]:
class InvalidCredentials(Exception):
    pass

def check_valid_request(r):
    if r.ok == False:
        if 'Invalid Credentials' in r.text:
            raise InvalidCredentials('The access token passed in the API request is invalid')
        else:
            raise Exception('The API request was unsuccesful')
            
params = {
    'access_token': access_token
}

r = requests.get(example_data_link, params=params)

check_valid_request(r)

zipped_files = zipfile.ZipFile(io.BytesIO(r.content))
zipped_files.extractall(f'{data_dir}/test')